# 前一段訓練程式看到衍伸函數
1. optimizer.zero_grad()<br>
2. with torch.no_grad()<br>

這些功能在幹嘛? (見下範例程式碼)

In [ ]:
model_mlp.train()
for batch_idx, (data, target) in enumerate(dataloader_train):
    data, target = data.to(device), target.to(device)
    # MLP
    optimizer_mlp.zero_grad()
    output_mlp = model_mlp(data)
    loss_mlp=loss(output_mlp,target)        
    loss_mlp.backward()
    optimizer_mlp.step()
    
with torch.no_grad():
    for data, target in dataloader_test:
        data, target = data.to(device), target.to(device)
         # MLP
        output_mlp = model_mlp(data)
        test_loss_mlp += loss(output_mlp, target)
        pred_mlp = output_mlp.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct_mlp += pred_mlp.eq(target.view_as(pred_mlp)).sum().item()

## 1. optimizer.zero_grad()
<font size=4 color=red> **此函數是用在將梯度歸0。** </font><br>

<font size=4 color=red> **- 什麼叫梯度歸0?** </font>

<font size=3 > 在每個batch在模型inference中，模型會將梯度同時也算出來，也就是此段程式碼 (output = model(data))

如果在pytorch不使用zero_grad()，也就是每次模型更新不歸0會發生什麼事情<br>
    
範例:  $y=x^2 → \frac{{\rm d}y}{{\rm d}x}=\frac{{\rm d}x^2 }{{\rm d}x}\ = 2x $<br>


In [1]:
import numpy as np
import torch
x = np.array([1.0,2.0,3.0])
x = torch.tensor(x, requires_grad=True)
print('x:{}'.format(x))
for i in range(5): 
    print('x:{}'.format(x.data))
    y = x**2
    y.sum().backward()
    # y.mean().backward()
    print('第{}次梯度:{}'.format(i+1, x.grad.data))

x:tensor([1., 2., 3.], dtype=torch.float64, requires_grad=True)
x:tensor([1., 2., 3.], dtype=torch.float64)
第1次梯度:tensor([2., 4., 6.], dtype=torch.float64)
x:tensor([1., 2., 3.], dtype=torch.float64)
第2次梯度:tensor([ 4.,  8., 12.], dtype=torch.float64)
x:tensor([1., 2., 3.], dtype=torch.float64)
第3次梯度:tensor([ 6., 12., 18.], dtype=torch.float64)
x:tensor([1., 2., 3.], dtype=torch.float64)
第4次梯度:tensor([ 8., 16., 24.], dtype=torch.float64)
x:tensor([1., 2., 3.], dtype=torch.float64)
第5次梯度:tensor([10., 20., 30.], dtype=torch.float64)


<font size=3> 
    所以可以發現如果我們沒有清掉梯度，在backward()部分梯度會將舊的梯度一直累加上去<br>
    所以需要將參數自行歸0。

In [2]:
# simple gradient
a = np.array([1.0, 2.0, 3.0])
a = torch.tensor(a, requires_grad=True)
for i in range(5): 
    c = a * a 
    out = c.sum()
    out.backward()
    print('*'*10)
    print('input:{}'.format(a.data))
    print('第{}次梯度:{}'.format(i+1, a.grad.data))
    a.grad.detach_()
    a.grad.zero_()


**********
input:tensor([1., 2., 3.], dtype=torch.float64)
第1次梯度:tensor([2., 4., 6.], dtype=torch.float64)
**********
input:tensor([1., 2., 3.], dtype=torch.float64)
第2次梯度:tensor([2., 4., 6.], dtype=torch.float64)
**********
input:tensor([1., 2., 3.], dtype=torch.float64)
第3次梯度:tensor([2., 4., 6.], dtype=torch.float64)
**********
input:tensor([1., 2., 3.], dtype=torch.float64)
第4次梯度:tensor([2., 4., 6.], dtype=torch.float64)
**********
input:tensor([1., 2., 3.], dtype=torch.float64)
第5次梯度:tensor([2., 4., 6.], dtype=torch.float64)


 ## <font color=blue> 梯度為歸0手寫範例
<font size=3>**梯度下降法**
<font size=3> \begin{gather*}
\theta^{(t+1)} = \theta^{(t)} - \eta \times f'(\theta^{(t)})
\end{gather*}
<font size=3> $f'(\theta)$為梯度，$\eta$為學習率(Learning rate)<br>
假設初始的$\theta^{(0)}$為10 <br>

1. 第一個batch的梯度$f'(\theta^{(0)})$為 2，$\eta$為1<br>
$$
\theta^{(1)} = \theta^{(0)} - \eta \times f'(\theta^{(0)}) = 10 - 1 \times 2 = 8
$$

  
2. 第二個batch的梯度$f'(\theta^{(1}))$為 1 <br>
如果梯度有歸0，則梯度更新則是
$$
\theta^{(2)} = \theta^{(1)} - \eta \times \color{red}{f'(\theta^{(1)})} = 8 - 1 \times 1 = 7
$$ 
如果梯度沒有歸0，則梯度會累加，更新方式則是
$$
\theta^{(2)} = \theta^{(1)} - \eta \times \color{red}{(f'(\theta^{(1)})+f'(\theta^{(0)}))} = 7 - 1 \times (1+2) = 4
$$
    
3. 第三個batch的梯度$f'(\theta^{(1}))$為 -1 <br>
如果梯度有歸0，則梯度更新則是
$$    
\theta^{(3)} = \theta^{(2)} - \eta \times \color{red}{f'(\theta^{(2)})} = 7 - 1 \times (-1) = 8
$$
如果梯度沒有歸0，則梯度會累加，更新方式則是
$$
\theta^{(3)} = \theta^{(2)} - \eta \times \color{red}{(f'(\theta^{(2)})+f'(\theta^{(1)})+f'(\theta^{(0)}))} = 4 - 1 \times (1+2-1) = 2
$$     
</font>   

Note:梯度不歸零不一定是用累積的方式，也能用平均的方式進行梯度融合。